# pix2pix Edge > RGB, tf.keras

In [ ]:
!pip install opencv-python #icrawler

# Crawling

In [ ]:
# import os
# import logging
# import shutil
# import requests
# from requests.packages.urllib3.exceptions import InsecureRequestWarning

# requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# TRAIN_DATA_PATH = 'train_images'

# if os.path.exists(TRAIN_DATA_PATH):
#     shutil.rmtree(TRAIN_DATA_PATH)

# os.makedirs(TRAIN_DATA_PATH, exist_ok=True)

# from icrawler.builtin import BingImageCrawler
# crawler = BingImageCrawler(storage={'root_dir': TRAIN_DATA_PATH}, parser_threads=2, downloader_threads=4, log_level=logging.CRITICAL)
# crawler.session.verify = False
# crawler.crawl(keyword="アカハライモリ", max_num=1000, min_size=(100, 100), max_size=None, file_idx_offset=0)

# # below is invalid 2020.5.6
# #from icrawler.builtin import GoogleImageCrawler
# #crawler = GoogleImageCrawler(storage={"root_dir": DIR_NAME})
# #crawler.crawl(keyword="アカハライモリ", max_num=100)

# Import

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

print(tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import cv2

# Config

In [ ]:
Input_height = 256
Input_width = 256
Input_channel = 1 # Gray

Output_channel = 3 # RGB

Base_path = 'drive/My Drive/Colab Notebooks/tf_keras/'

# Model

In [ ]:
def UNet():
    def unet_encoder(x, filters, ksize=4, bn=False):
        initializer = tf.random_normal_initializer(0., 0.02)
        x = tf.keras.layers.Conv2D(filters, ksize, strides=2, padding='same', use_bias=False, kernel_initializer=initializer)(x)
        #x = tf.keras.layers.ReLU()(x)
        if bn:
            x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU()(x)
        return x

    def unet_decoder(x, filters, ksize=4, do=False):
        initializer = tf.random_normal_initializer(0., 0.02)
        x = tf.keras.layers.Conv2DTranspose(filters, ksize, strides=2, padding='same', use_bias=False, kernel_initializer=initializer)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if do:
            x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.ReLU()(x)
        return x

    conv_dim = 16

    _input = tf.keras.layers.Input(shape=[Input_height, Input_width, Input_channel])
    x_enc1 = unet_encoder(_input, conv_dim, bn=False)
    x_enc2 = unet_encoder(x_enc1, conv_dim * 2)
    x_enc3 = unet_encoder(x_enc2, conv_dim * 4)
    x_enc4 = unet_encoder(x_enc3, conv_dim * 8)
    x_enc5 = unet_encoder(x_enc4, conv_dim * 8)
    x_enc6 = unet_encoder(x_enc5, conv_dim * 8)
    x_enc7 = unet_encoder(x_enc6, conv_dim * 8)
    x_enc8 = unet_encoder(x_enc7, conv_dim * 8)

    # up sample
    x_dec7 = unet_decoder(x_enc8, conv_dim * 8, do=True)
    x_dec7 = tf.keras.layers.concatenate([x_dec7, x_enc7])

    x_dec6 = unet_decoder(x_dec7, conv_dim * 8, do=True)
    x_dec6 = tf.keras.layers.concatenate([x_dec6, x_enc6])

    x_dec5 = unet_decoder(x_dec6, conv_dim * 8, do=True)
    x_dec5 = tf.keras.layers.concatenate([x_dec5, x_enc5])

    x_dec4 = unet_decoder(x_dec5, conv_dim * 8)
    x_dec4 = tf.keras.layers.concatenate([x_dec4, x_enc4])

    x_dec3 = unet_decoder(x_dec4, conv_dim * 4)
    x_dec3 = tf.keras.layers.concatenate([x_dec3, x_enc3])

    x_dec2 = unet_decoder(x_dec3, conv_dim * 2)
    x_dec2 = tf.keras.layers.concatenate([x_dec2, x_enc2])

    x_dec1 = unet_decoder(x_dec2, conv_dim)
    x_dec1 = tf.keras.layers.concatenate([x_dec1, x_enc1])
    
    x_out = tf.keras.layers.Conv2DTranspose(Output_channel, 4, strides=2, padding='same', activation='tanh')(x_dec1)
    return tf.keras.Model(inputs=_input, outputs=x_out)

In [ ]:
def Discriminator():
    def disc_encoder(x, filters, ksize, bn=True):
        x = tf.keras.layers.Conv2D(filters, ksize, strides=2, padding='same', use_bias=False)(x)
        if bn:
            x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU()(x)
        return x

    _input1 = tf.keras.layers.Input(shape=[Input_height, Input_width, Input_channel], name='input1')
    _input2 = tf.keras.layers.Input(shape=[Input_height, Input_width, Output_channel], name='input2')
    
    x = tf.keras.layers.concatenate([_input1, _input2])
    x = disc_encoder(x, 64, 5, bn=False)
    x = disc_encoder(x, 128, 5)
    x = disc_encoder(x, 256, 5)
    x = disc_encoder(x, 512, 5)
    x = tf.keras.layers.Conv2D(1, 5, strides=1, padding='same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1)(x)

    return tf.keras.Model(inputs=[_input1, _input2], outputs=x)

In [ ]:
# def UNet():
#     def UNet_block_downSampling(x, filters, size, name, apply_batchnorm=False):
#         x = tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', use_bias=False, name=name + '_conv')(x)
#         #x = tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=tf.random_normal_initializer(0., 0.02), 
#         #                           use_bias=False, name=name + '_conv')(x)
#         #x = tf.keras.layers.LeakyReLU(name=name + '_leakyReLU')(x) if apply_batchnorm else x
#         x = tf.keras.layers.ReLU(name=name + '_ReLU')(x)
#         x = tf.keras.layers.BatchNormalization(name=name + '_bn')(x)
        
#         return x

#     def UNet_block_upSampling(x, filters, size, name, apply_dropout=False):
#         x = tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same', use_bias=False, name=name + '_transposedConv')(x)
#         #x = tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer=tf.random_normal_initializer(0., 0.02),
#         #                                    use_bias=False, name=name + '_transposedConv')(x)
#         #x = tf.keras.layers.ReLU(name=name + '_ReLU')(x)
#         x = tf.keras.layers.BatchNormalization(name=name + '_bn')(x)
#         x = tf.keras.layers.Dropout(0.5, name=name + 'dropout')(x) if apply_dropout else x
        
#         return x

#     stride_N = int(np.log(Input_height) / np.log(2))

#     x_encoders = []

#     _input = tf.keras.layers.Input(shape=[Input_height, Input_width, Input_channel])
#     # down sample
#     x = _input
#     for i in range(1, stride_N + 1):
#         x = UNet_block_downSampling(x, min(64 ** i, 512), 3, name='Encoder{}'.format(i))
#         x_encoders.append(x)
#     """
#     x_encoder1 = UNet_block_downSampling(_input, 64, 4, apply_batchnorm=False, name='Encoder1') # (bs, 128, 128, 64)
#     x_encoder2 = UNet_block_downSampling(x_encoder1, 128, 4, name='Encoder2') # (bs, 64, 64, 128)
#     x_encoder3 = UNet_block_downSampling(x_encoder2, 256, 4, name='Encoder3') # (bs, 32, 32, 256)
#     x_encoder4 = UNet_block_downSampling(x_encoder3, 512, 4, name='Encoder4') # (bs, 16, 16, 512)
#     x_encoder5 = UNet_block_downSampling(x_encoder4, 512, 4, name='Encoder5') # (bs, 8, 8, 512)
#     x_encoder6 = UNet_block_downSampling(x_encoder5, 512, 4, name='Encoder6') # (bs, 4, 4, 512)
#     x_encoder7 = UNet_block_downSampling(x_encoder6, 512, 4, name='Encoder7') # (bs, 2, 2, 512)
#     x_encoder8 = UNet_block_downSampling(x_encoder7, 512, 4, name='Encoder8') # (bs, 1, 1, 512)
#     """

#     for i in range(stride_N - 1, 0, -1):
#         x = UNet_block_upSampling(x, min(64 ** i, 512), 3, name='Decoder{}'.format(i - 1))
#         x = tf.keras.layers.concatenate([x, x_encoders[i - 1]])

#     # up sample
#     """
#     x_decoder7 = UNet_block_upSampling(x_encoder8, 512, 4, apply_dropout=True, name='Decoder7') # (bs, 2, 2, 512)
#     x_concat7 = tf.keras.layers.Concatenate()([x_decoder7, x_encoder7]) # (bs, 2, 2, 1024)
#     x_decoder6 = UNet_block_upSampling(x_concat7, 512, 4, apply_dropout=True, name='Decoder6') # (bs, 4, 4, 512)
#     x_concat6 = tf.keras.layers.Concatenate()([x_decoder6, x_encoder6]) # (bs, 4, 4, 1024)
#     x_decoder5 = UNet_block_upSampling(x_concat6, 512, 4, apply_dropout=True, name='Decoder5') # (bs, 8, 8, 512)
#     x_concat5 = tf.keras.layers.Concatenate()([x_decoder5, x_encoder5]) # (bs, 8, 8, 1024)
#     x_decoder4 = UNet_block_upSampling(x_concat5, 512, 4, name='Decoder4') # (bs, 16, 16, 512)
#     x_concat4 = tf.keras.layers.Concatenate()([x_decoder4, x_encoder4]) # (bs, 16, 16, 1024)
#     x_decoder3 = UNet_block_upSampling(x_concat4, 256, 4, name='Decoder3') # (bs, 32, 32, 256)
#     x_concat3 = tf.keras.layers.Concatenate()([x_decoder3, x_encoder3]) # (bs, 32, 32, 512)
#     x_decoder2 = UNet_block_upSampling(x_concat3, 128, 4, name='Decoder2') # (bs, 64, 64, 128)
#     x_concat2 = tf.keras.layers.Concatenate()([x_decoder2, x_encoder2]) # (bs, 64, 64, 256)
#     x_decoder1 = UNet_block_upSampling(x_concat2, 64, 4, name='Decoder1') # (bs, 128, 128, 64)
#     x_concat1 = tf.keras.layers.Concatenate()([x_decoder1, x_encoder1]) # (bs, 128, 128, 128)
#     """

#     x_output = tf.keras.layers.Conv2DTranspose(Output_channel, 3, strides=2, padding='same', activation='tanh')(x)
#     #x_output = tf.keras.layers.Conv2DTranspose(out_channel, 3, strides=2, padding='same',
#     #                                     kernel_initializer=tf.random_normal_initializer(0., 0.02), activation='tanh')(x)
#     return tf.keras.Model(inputs=_input, outputs=x_output)

# def Discriminator():
#     def Discriminator_block_downSampling(x, filters, size, name, apply_batchnorm=False):
#         x = tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', use_bias=False, name=name + '_conv')(x)
#         #x = tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=tf.random_normal_initializer(0., 0.02), 
#         #                           use_bias=False, name=name + '_conv')(x)
#         #x = tf.keras.layers.BatchNormalization(name=name + '_bn')(x)
#         x = tf.keras.layers.LeakyReLU(name=name + '_leakyReLU')(x) if apply_batchnorm else x
#         return x

#     stride_N = int(np.log(Input_height) / np.log(2))

#     _input1 = tf.keras.layers.Input(shape=[Input_height, Input_width, Input_channel], name='input1')
#     _input2 = tf.keras.layers.Input(shape=[Input_height, Input_width, Input_channel], name='input2')
    
#     x = tf.keras.layers.concatenate([_input1, _input2]) # (bs, 256, 256, channels*2)

#     for i in range(1, stride_N):
#         x = Discriminator_block_downSampling(x, min(64 ** i, 512), 5, name='D{}'.format(i))
    
#     x = tf.keras.layers.Conv2D(1, 2, strides=1, padding='same')(x)
#     #x = tf.keras.layers.Conv2D(1, 2, strides=1, padding='same', kernel_initializer=tf.random_normal_initializer(0., 0.02))(x) # (bs, 30, 30, 1)

#     return tf.keras.Model(inputs=[_input1, _input2], outputs=x)

# Utility

In [ ]:
class BatchGenerator():
    def __init__(self, data_size, batch_size, shuffle=True):
        self.data_size = data_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.mbi = 0 # index for iteration
        self.inds = np.arange(data_size)
        if shuffle:
            np.random.shuffle(self.inds)

    def __call__(self):
        if self.mbi + self.batch_size > self.data_size:
            inds = self.inds[self.mbi:]
            if self.shuffle:
                np.random.shuffle(self.inds)
            inds = np.hstack((inds, self.inds[ : (self.batch_size - (self.data_size - self.mbi))]))
            self.mbi = self.batch_size - (self.data_size - self.mbi)
        else:
            inds = self.inds[self.mbi : self.mbi + self.batch_size]
            self.mbi += self.batch_size
        return inds

In [ ]:
def get_image(paths, aug_hflip=False, aug_vflip=False, aug_shift=False, aug_rot=False, aug_affine=False):
    xs = []
    xs_edge = []
    
    for path in paths:
        x = cv2.imread(path.replace("/akahara_imori_seg/", "/akahara_imori/").replace(".png", ".jpg"))
        x = x[..., ::-1] # BGR > RGB
        x = cv2.resize(x, (Input_width, Input_height)) # resize

        # get Edge
        x_gray = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        x_gray = cv2.resize(x_gray, (Input_width, Input_height)) # resize

        # if aug_hflip: # augmentation horizontal flip
        #     if np.random.random() < 0.5:
        #         x = x[:, ::-1]
        #         x_seg = x_seg[:, ::-1]
        

        #x_gray = cv2.cvtColor(x_seg, cv2.COLOR_BGR2GRAY)
        x_edge = cv2.Canny(x_gray, 100, 200)

        kernel = np.ones((2, 2), np.uint8)
        x_edge = cv2.dilate(x_edge, kernel, iterations=1)
        x_edge[x_edge > 0] = 255

        # random drop pixel
        # ind = np.random.rand(x_edge.shape[0], x_edge.shape[1])
        # ind = (ind // 0.6).astype(bool)
        # x_edge[ind] = 0
        # random add pixel
        # ind = np.random.rand(x_edge.shape[0], x_edge.shape[1])
        # ind = (ind // 0.9).astype(bool)
        # x_edge[ind] = 255


        if aug_hflip:
            if np.random.random() < 0.5:
                x = x[:, ::-1]
                x_edge = x_edge[:, ::-1]
                x_gray = x_gray[:, ::-1]

        if aug_vflip:
            if np.random.random() < 0.5:
                x = x[::-1]
                x_edge = x_edge[::-1]
                x_gray = x_gray[::-1]

        if aug_shift:
            M = np.float32([[1, 0, np.random.randint(-Input_width // 3, Input_width // 3)], [0, 1, np.random.randint(-Input_height // 3, Input_height // 3)]])
            x = cv2.warpAffine(x, M, (x.shape[1], x.shape[0]))
            x_edge = cv2.warpAffine(x_edge, M, (x_edge.shape[1], x_edge.shape[0]))
            x_gray = cv2.warpAffine(x_gray, M, (x_edge.shape[1], x_edge.shape[0]))

        if aug_rot:
            M = cv2.getRotationMatrix2D((x.shape[1]/2, x.shape[0]/2), np.random.randint(-45, 45), 1)
            x = cv2.warpAffine(x, M, (x.shape[1], x.shape[0]))
            x_edge = cv2.warpAffine(x_edge, M, (x_edge.shape[1], x_edge.shape[0]))
            x_gray = cv2.warpAffine(x_gray, M, (x_edge.shape[1], x_edge.shape[0]))

        if aug_affine:
            dis_h = Input_height // 4
            dis_w = Input_width // 4
            p_original = np.float32([[0, 0], [x.shape[0], 0], [0, x.shape[1]], [x.shape[0], x.shape[1]]])
            p_trans = np.float32(
                [[np.random.randint(dis_w), np.random.randint(dis_h)], 
                [x.shape[1] - np.random.randint(dis_w), np.random.randint(dis_h)], 
                [np.random.randint(dis_w), x.shape[0] - np.random.randint(dis_h)], 
                [x.shape[1] - np.random.randint(dis_w), x.shape[0] - np.random.randint(dis_h)]])
            M = cv2.getPerspectiveTransform(p_original, p_trans)
            x = cv2.warpPerspective(x, M, (x.shape[1], x.shape[0]))
            x_edge = cv2.warpPerspective(x_edge, M, (x.shape[1], x.shape[0]))
            x_gray = cv2.warpAffine(x_gray, M, (x_edge.shape[1], x_edge.shape[0]))

        # masking
        x_edge[x_edge > 0] = 255
        x[x_gray == 0] = 255

        xs.append(x)
        xs_edge.append(x_edge)
                
    xs = np.array(xs, dtype=np.float32)
    xs_edge = np.array(xs_edge, dtype=np.float32)

    xs_edge = np.expand_dims(xs_edge, axis=-1)

    # normalize
    xs = xs / 127.5 - 1
    xs_edge = xs_edge / 255

    return xs, xs_edge

In [ ]:
# get image path
#img_paths = np.array(glob(f'{TRAIN_DATA_PATH}/*.jpg'))
img_paths = np.array(glob(f'drive/My Drive/Colab Notebooks/Dataset/imori/akahara_imori_seg/*.png'))
data_num = len(img_paths)
print(f'data num >> {data_num}')

In [ ]:
# sample plot
for i in range(5):
    plt.subplot(1, 2, 1)
    plt.imshow(get_image(img_paths[[i]])[1][0, ..., 0])
    cv2.imwrite(f'drive/My Drive/Colab Notebooks/tf_keras/sample_input{i+1}.png', (get_image(img_paths[[i]])[1][0, ..., 0] * 255).astype(np.uint8))
    plt.subplot(1, 2, 2)
    plt.imshow(get_image(img_paths[[i]])[0][0] * 0.5 + 0.5)
    plt.show()

In [ ]:
x = cv2.imread("drive/My Drive/Colab Notebooks/Dataset/imori/akahara_imori/000041.jpg")[..., ::-1]
for i in range(10):
    plt.subplot(1, 2, 1)
    plt.imshow(x)
    # M = np.float32([[1, 0, np.random.randint(-Input_width // 3, Input_width // 3)], [0, 1, np.random.randint(-Input_height // 3, Input_height // 3)]])
    # x2 = cv2.warpAffine(x, M, (x.shape[1], x.shape[0]))
    # M = cv2.getRotationMatrix2D((x.shape[1]/2, x.shape[0]/2), np.random.randint(-45, 45), 1)
    # x2 = cv2.warpAffine(x, M, (x.shape[1], x.shape[0]))
    p_original = np.float32([[0, 0], [x.shape[0], 0], [0, x.shape[1]], [x.shape[0], x.shape[1]]])
    p_trans = np.float32(
        [[np.random.randint(30), np.random.randint(30)], 
         [x.shape[0] - np.random.randint(30), np.random.randint(30)], 
         [np.random.randint(30), x.shape[1] - np.random.randint(30)], 
         [x.shape[0] - np.random.randint(30), x.shape[1] - np.random.randint(30)]])
    M = cv2.getPerspectiveTransform(p_original, p_trans)
    x2 = cv2.warpPerspective(x, M, (x.shape[1], x.shape[0]))
    plt.subplot(1, 2 ,2)
    plt.imshow(x2)
    plt.show()

# Train

In [ ]:
# model
G = UNet()
print('Generator loaded')
D = Discriminator()
print('Discriminator loaded')

In [ ]:
# optimizer
G_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
D_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
Loss_Lambda = 10.

@tf.function
def train_step(x, target):
    with tf.GradientTape() as G_tape, tf.GradientTape() as D_tape:
        Gx = G(x, training=True)
        D_real = D([x, target], training=True)
        D_fake = D([x, Gx], training=True)

        # Generator loss
        G_loss_fake = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(D_fake), D_fake)
        G_loss_L1 = tf.reduce_mean(tf.abs(target - Gx))
        G_loss = G_loss_fake + Loss_Lambda * G_loss_L1

        # Discriminator loss
        D_loss_real = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(D_real), D_real)
        D_loss_fake = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(D_fake), D_fake)
        D_loss = D_loss_real + D_loss_fake

    G_gradients = G_tape.gradient(G_loss, G.trainable_variables)
    D_gradients = D_tape.gradient(D_loss, D.trainable_variables)

    G_opt.apply_gradients(zip(G_gradients, G.trainable_variables))
    D_opt.apply_gradients(zip(D_gradients, D.trainable_variables))

    return {'G_loss' : G_loss, 'G_loss_fake' : G_loss_fake, 'G_loss_L1' : G_loss_L1,
            'D_loss' : D_loss, 'D_loss_real' : D_loss_real, 'D_loss_fake' : D_loss_fake,
            'Gx' : Gx
            }

In [ ]:
# training parameter
Iteration = 30_000
Minibatch = 8

In [ ]:
# training
pbar = ''


# buffer
# all_Xs, all_Xs_edge = get_image(img_paths)

# all_Xs_2, all_Xs_edge_2 = get_image(img_paths, aug_hflip=True)
# all_Xs_3, all_Xs_edge_3 = get_image(img_paths, aug_vflip=True)

# all_Xs = np.vstack([all_Xs, all_Xs_2, all_Xs_3])
# all_Xs_edge = np.vstack([all_Xs_edge, all_Xs_edge_2, all_Xs_edge_3])

print('train start')

# training
batch_gen = BatchGenerator(len(img_paths), Minibatch) # all data num, minibatch

for ite in range(Iteration):
    # batch generate
    mb_inds = batch_gen()
    Xs, Xs_edge = get_image(img_paths[mb_inds], aug_hflip=True, aug_vflip=True, aug_shift=True, aug_rot=True)
    # Xs = all_Xs[mb_inds]
    # Xs_edge = all_Xs_edge[mb_inds]

    # train step
    res_dict = train_step(Xs_edge, Xs)

    # show progress bar
    pbar = pbar + f'|{ite + 1}'  if (ite + 1) % 10 == 0 else pbar + '|'
    print(f'\r{pbar}', end='')
    
    # show loss
    if (ite + 1) % 100 == 0:
        print('\r' + ' ' * len(pbar), end='')
        print(f"\riter : {ite + 1} G_Loss : {res_dict['G_loss']:.4f} fake : {res_dict['G_loss_fake']:.4f} , L1 : {res_dict['G_loss_L1']:.4f}, D_Loss : {res_dict['D_loss']:.4f} fake : {res_dict['D_loss_fake']:.4f} , real : {res_dict['D_loss_real']:.4f}")
        pbar = ''

    # show sample
    if (ite + 1) % 1_000 == 0:
        Gxs = res_dict['Gx'] * 0.5  + 0.5
        plt.subplot(1, 4, 1); plt.imshow(Xs_edge[0, ..., 0]); plt.title('input 1')
        plt.subplot(1, 4, 2); plt.imshow(Gxs[0]); plt.title('output 1')
        plt.subplot(1, 4, 3); plt.imshow(Xs_edge[1, ..., 0]); plt.title('input 2')
        plt.subplot(1, 4, 4); plt.imshow(Gxs[1]); plt.title('output 2')
        plt.show()

    if (ite + 1) % 10_000 == 0:
        G.save(f'{Base_path}/pix2pix_edge2akahara_g_iter{ite + 1}_model.h5')
        # D.save_weights(f'{Base_path}/pix2pix_edge2akahara_d_iter{ite + 1}.h5')


G.save_weights(f'{Base_path}/pix2pix_edge2akahara_iter{ite + 1}.h5')
G.save(f'{Base_path}/pix2pix_edge2akahara_g_iter{ite + 1}_model.h5')
D.save_weights(f'{Base_path}/pix2pix_edge2akahara_d_iter{ite + 1}.h5')

In [ ]:
# G.load_weights(f'{Base_path}/pix2pix_edge2rgb_g_iter100000.h5')
# G.save(f'{Base_path}/pix2pix_edge2rgb_g_iter100000_ver2.h5')